# LangChain + OpenAI RAG Proof of Concept
## Overview
* Collates provided sources into local vector store
* Persists store for future recovery
* Identifies relevant chunks and constructs prompt to send to OpenAI
### Dependencies
* User account for OpenAI and LangSmith
* API keys from OpenAI and LangSmith
* Main python dependencies:
  * LangChain: langchain-openai langchain-core langchain-text-splitters langchain-community langgraph langchain[openai]
  * GDrive: google-api-python-client google-auth-httplib2 google-auth-oauthlib langchain-google-community
  * Web: bs4 unstructured selenium
* Add the following to `~/.bash_profile` or `~/.zprofile` if `~/.zshrc`
```
export LANGSMITH_TRACING=true
export LANGSMITH_ENDPOINT="https://{langsmith_region}.api.smith.langchain.com"
export LANGSMITH_API_KEY="{your_langsmith_api_key}"
export OPENAI_API_KEY="{your_openai_api_key}"
```

In [1]:
import getpass
import sys
import os
from pathlib import Path

# Set up environment variables if not already set
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

if not os.environ.get("LANGSMITH_API_KEY"):
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter API key for LangSmith: ")

os.environ["USER_AGENT"] = "rag-ingestion-pipeline/1.0"

# Add the project root to the Python path
project_root = Path().absolute().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

In [2]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from src.document_loaders import (
    load_web_document,
    load_pdf_document,
    load_google_drive_document,
    load_dynamic_web_document,
    load_carbon_intensity
)
from src.vector_store import VectorStoreManager
from src.rag_pipeline import RAGPipeline

# Load all documents first
project_id = "my_first_project"
all_docs = load_dynamic_web_document(
    ["https://www.greatyellow.earth", "https://www.greatyellow.earth/about"],
    project_id=project_id
) + load_pdf_document(
    [
        "/Users/jsouthin/Documents/Joe Southin - CV 2025 (A4).pdf",
        "/Users/jsouthin/Downloads/joe southin cv 2016.pdf"
    ],
    project_id=project_id
) + load_google_drive_document(
    ["1xCW-ZiquUxwBLpMTn9kL6WrvPlfeBYpDuicQhiIR81w"],
    "/Users/jsouthin/Downloads/lively-tensor-432422-c0-407d22e805d2.json",
    project_id=project_id
) + load_carbon_intensity(project_id=project_id)

print(f"Loaded {len(all_docs)} documents")

/Users/jsouthin/Projects/rag/src/document_loaders.py:147: LangChainDeprecationWarning: The class `GoogleDriveLoader` was deprecated in LangChain 0.0.32 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-google-community package and should be used instead. To use it run `pip install -U :class:`~langchain-google-community` and import as `from :class:`~langchain_google_community import GoogleDriveLoader``.
  loader = GoogleDriveLoader(


Loaded 8 documents


In [3]:
# Initialize models
llm = ChatOpenAI()
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# Initialize vector store manager
vector_store = VectorStoreManager(embedding_model)

# Try to load existing store or create new one
vector_store_path = "./data/faiss_index"
try:
    vector_store.load_store(
        vector_store_path,
        allow_dangerous_deserialization=True
    )
    print(f"Loaded existing vector store with {vector_store.document_count} documents")
    remaining_docs = all_docs  # All docs need to be added if we loaded existing store
except FileNotFoundError:
    print("No existing vector store found. Will create new one.")
    # Initialize with first document
    vector_store.init_store([all_docs[0]])
    # Remove the first document since it's already added
    remaining_docs = all_docs[1:]
    print("Initialized new vector store with first document")

# Initialize RAG pipeline
rag = RAGPipeline(vector_store, llm)

# Add remaining documents to pipeline
if remaining_docs:
    rag.add_documents(remaining_docs)

# Save updated vector store
vector_store.save_store(vector_store_path, force=True)

Vector store loaded from './data/faiss_index'.
Loaded existing vector store with 436 documents
Vector store updated with 111 new documents.
Vector store saved to './data/faiss_index'.


In [4]:
# Test some queries
questions = [
    "explain the differences between a type 1, 2, and 3 model",
    "why the name Great Yellow?",
    "What is joe southin's biggest accomplishment?"
]

for question in questions:
    print(f"Q: {question}")
    answer = rag.query(question, project_id)
    print(f"A: {answer}\n")

Q: explain the differences between a type 1, 2, and 3 model
A: The type 3 model is known as the Campaign id OHE model, where campaign id and client id columns are one hot encoded and used as features. It can generate inference for campaigns present in the previous week's training set. Inference for campaigns not in the training set can be done using the Type 2 model, also known as the Customer id OHE model, where the pca_id is encoded and client_id and campaign_id columns are dropped.

Q: why the name Great Yellow?
A: The name Great Yellow is inspired by the Great Yellow butterfly, which serves as a symbol of the urgent need to protect biodiversity. Great Yellow Ltd aims to showcase the economic value of thriving natural systems, drawing inspiration from the Great Yellow butterfly.

Q: What is joe southin's biggest accomplishment?
A: Joe Southin's biggest accomplishment is developing an ML-powered budget management solution that optimized spend allocation and increased customer retenti

In [5]:
# Repeat some queries, but this time change project such that the context is no longer available
questions = [
    "explain the differences between a type 1, 2, and 3 model",
    "why the name Great Yellow?",
    "What is joe southin's biggest accomplishment?"
]

for question in questions:
    print(f"Q: {question}")
    answer = rag.query(question, "some_other_project")
    print(f"A: {answer}\n")

Q: explain the differences between a type 1, 2, and 3 model
A: I'm sorry, I don't have enough information to answer that question.

Q: why the name Great Yellow?
A: The name "Great Yellow" might refer to the color yellow being associated with positivity, energy, and happiness. It could also be a creative or unique name chosen by the individual or company for branding purposes. Without further information, it is difficult to determine the specific reason behind the name "Great Yellow."

Q: What is joe southin's biggest accomplishment?
A: I'm sorry, I don't have enough information to determine Joe Southin's biggest accomplishment.



In [6]:
rag.query('whats the carbon intensity index?', project_id)

'The carbon intensity index for the specified half-hour period is 154, with a forecasted value of 191. The index is categorized as moderate based on these values.'